# L3: Debug DSPy Agent with MLflow Tracing

Run mlflow:


```
docker pull ghcr.io/mlflow/mlflow:latest
docker run -d -p 5000:5000 ghcr.io/mlflow/mlflow:latest mlflow server --host 0.0.0.0
```

In [6]:
from dotenv import load_dotenv
import mlflow
assert(load_dotenv())
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Debug DSPy Agent")
mlflow.dspy.autolog()

ModuleNotFoundError: No module named 'mlflow'